PROYECTO ETL

In [1]:
import requests as req
import os
from dotenv import load_dotenv
import base64
from requests import post,get
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()
client_id=os.getenv('CLIENT_ID')
client_secret=os.getenv('CLIENT_SECRET')

In [3]:
#para entrar a la api
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + auth_base64,
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {'grant_type': 'client_credentials'}
    result = post(url, headers=headers, data=data)
    
    json_result = result.json()
    token = json_result.get('access_token', None) 
    return token

token = get_token()


In [4]:
def get_auth_header(token):
    return {'Authorization':'Bearer '+ token}


In [5]:
#Funcion para buscar unicamente los id de los albums
def search_for_album_id(token, album_name):
    url = 'https://api.spotify.com/v1/search'
    headers = {'Authorization': f'Bearer {token}'}
    params = {
        'q': album_name,
        'type': 'album',
        'limit': 1
    }

    response = req.get(url, headers=headers, params=params)
    json_result = response.json()

    albums_items = json_result.get('albums', {}).get('items', [])
    if albums_items:

        return albums_items[0].get('id')
    else:
        return None

# Importo el database que limpie para sacar los albums id

In [6]:
#Creo una nueva tabla para las canciones de los albums
canciones = pd.read_csv('../database/csv_limpios/albums.csv')

In [7]:
canciones['album_id'] = canciones['Album'].apply(lambda x: search_for_album_id(token, x))

In [8]:
canciones = canciones.drop(columns=['Year', 'Ranking','Worldwide Sales','CDs','Genre'])


In [9]:
lista_id = canciones['album_id'].tolist()

In [10]:
# Funcion para obtener las canciones de los albums
def cancion(album_id, token):
    headers = {'Authorization': f'Bearer {token}'}
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    try:
        response = req.get(url, headers=headers)
        album_tracks = response.json().get('items', []) 
        return [track['name'] for track in album_tracks]  
    except:
        pass

In [11]:
#Aplico la funcion para todos los albums_id
canciones['Pistas'] = canciones['album_id'].apply(lambda x: cancion(x, token))

In [12]:
canciones = canciones.reset_index().rename(columns={'index': 'canciones_id'})

In [13]:
#Desarmo la lista
canciones['Pistas'] = canciones['Pistas'].apply(lambda x: ', '.join(x))

In [14]:
canciones

,canciones_id,Artist,Album,Tracks,album_id,Pistas
0,0,Madonna,The Immaculate Collection,17,5MzwGwnO9gkh0z6Nl4FF8h,"Holiday, Lucky Star - US ReMix Version, Border..."
1,1,New_Kids_on_the_Block,Step By Step,12,4dCdnfD0shXGuHtyTOtx8q,"Step by Step, Tonight, Baby, I Believe In You,..."
2,2,Garth Brooks,No Fences,10,2l8LCtgshAUZlSZbLAzWr4,"don't tell anyone, i hope i never have a daugh..."
3,3,Metallica,Metallica,12,55fq75UfkYbGMq4CncCtOH,"Enter Sandman (Remastered), Sad But True (Rema..."
4,4,Nirvana_(banda),Nevermind,13,2guirTSEqLizK7j9i1MTTZ,"Smells Like Teen Spirit, In Bloom, Come As You..."
...,...,...,...,...,...,...
91,91,BTS,Be,8,0P3oVJBFOv3TDXlYRhGL7s,"Real Life, Losers, Tell Your Friends, Often, T..."
92,92,Pop Smoke,"Shoot For The Stars, Aim For The Moon",19,7e7t0MCrNDcJZsPwUKjmOc,"Bad Bitch From Tokyo (Intro), Aim For The Moon..."
93,93,Adele,30,12,21jF5jlMtzo94wbxmJ18aa,"Strangers By Nature, Easy On Me, My Little Lov..."
94,94,Olivia Rodrigo,Sour,11,6s84u2TUpR3wdUv4NgKA2j,"brutal, traitor, drivers license, 1 step forwa..."


# Agrego una columna de popularidad a la tabla de albums usando la api


In [15]:
albums2 = pd.read_csv('../database/csv_limpios/albums.csv')

In [16]:
#Primero agrego los album_id a la tabla
albums2['album_id'] = albums2['Album'].apply(lambda x: search_for_album_id(token, x))

In [17]:
# Popularidad de los albums del ranking hoy en dia, esta entre 0 y 100 siendo 100 el mas popular
#La popularidad se calcula por la cantidad de reproducciones de las pistas del álbum y qué tan recientes son esas reproducciones.
def popularity(token, album_id):
    
    url = f'https://api.spotify.com/v1/albums/{album_id}'
    headers = {'Authorization': f'Bearer {token}'}

    response = req.get(url, headers=headers)
    album_details = response.json()

    popularity = album_details.get('popularity')

    return popularity

albums2['popularity'] = albums2['album_id'].apply(lambda x: popularity(token, x))

In [18]:
albums2 = albums2.reset_index().rename(columns={'index': 'albums_id'})

# Exporto canciones 

In [20]:
canciones.to_csv('../database/csv_limpios/canciones.csv', index=False)

albums2.to_csv('../database//csv_limpios/albums_completo.csv', index=False)